In [1]:
import fitz
import pysubs2
from bs4 import BeautifulSoup
from sub_PDF_location import get_time_location
from PIL import Image
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip


In [2]:
def draw_line(pdf_file_path, page_num, top, left, length,height,color=(1,1,0), dpi=300):
    pdf_document = fitz.open(pdf_file_path)
    page=pdf_document[page_num]
    # 绘制从左下，到右下的直线
    x0=left
    y0=top+height
    x1=left+length
    y1=top+height

    shape=page.new_shape()
    shape.draw_line((x0,y0),(x1,y1)) 
    shape.finish(color=color)
    shape.commit()
    pix = page.get_pixmap(matrix=fitz.Matrix(dpi/72, dpi/72))
    return pix 


def pixmap_to_cv_image(pixmap):
    # 获取图像数据
    image_data = np.frombuffer(pixmap.samples, dtype=np.uint8)
    
    # 将图像数据重新调整为正确的形状
    if pixmap.alpha:
        shape = (pixmap.height, pixmap.width, 4)
    else:
        shape = (pixmap.height, pixmap.width, 3)
    image_data = image_data.reshape(shape)

    # 将图像从RGBA转换为BGR格式
    if pixmap.alpha:
        image_data = cv2.cvtColor(image_data, cv2.COLOR_RGBA2BGR)
    else:
        image_data = cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR)

    return image_data
    

In [3]:
if __name__ == '__main__':
    pdf_file_path = 'test_pdf/test2.pdf'
    vtt_file_path = 'test_pdf/test.vtt'
    start_page = 1
    end_page = 2
    time_location = get_time_location(pdf_file_path, vtt_file_path, start_page, end_page)
    images=[]
    timestamps=[]
    for i, t in enumerate(time_location[:20]):
        # print(t)
        pix=draw_line(pdf_file_path, start_page+t['page_num'], t['top'], t['left'], t['line_length']/2*0.9,t['line_height'])
        # pix.save(f"test_pdf/test{i:03d}.png")
        cv2_img=pixmap_to_cv_image(pix)
        images.append(cv2_img)
        timestamps.append((t['start_time']/1000,t['end_time']/1000))
    


In [4]:
def create_video_from_images(images, timestamps, output_video, frame_rate):
    # 获取图像尺寸
    height, width, layers = images[0].shape

    # 创建VideoWriter对象，用于生成视频文件
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # 使用mp4格式
    video = cv2.VideoWriter(output_video, fourcc, frame_rate, (width, height))

    # 将图像添加到视频中
    for start_time, end_time in timestamps:
        duration = end_time - start_time
        total_frames = int(duration * frame_rate)

        for img in images:
            for _ in range(total_frames):
                video.write(img)

    # 释放VideoWriter对象并关闭窗口
    video.release()
    cv2.destroyAllWindows()

def add_audio_to_video(video_path, audio_path, output_path):
    video = VideoFileClip(video_path)
    audio = AudioFileClip(audio_path)
    video_with_audio = video.set_audio(audio)
    video_with_audio.write_videofile(output_path)

# # 示例使用
# images = [cv2.imread('image1.jpg'), cv2.imread('image2.jpg'), cv2.imread('image3.jpg')] # 内存中存储的图像列表
# timestamps = [(100, 150), (152, 160), (165, 180)] # 时间戳列表，单位为秒
output_video = 'output_video_no_audio.mp4' # 输出无音频的视频文件名
final_video = 'output_video_with_audio.mp4' # 输出带音频的视频文件名
frame_rate = 30 # 视频的帧率
audio_path = 'test_pdf/test.mp3' # 音频文件路径

create_video_from_images(images, timestamps, output_video, frame_rate)
add_audio_to_video(output_video, audio_path, final_video)


Moviepy - Building video output_video_with_audio.mp4.
MoviePy - Writing audio in output_video_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video_with_audio.mp4



Moviepy - Done !
Moviepy - video ready output_video_with_audio.mp4
